In this live coding exercise, we will implement the formulation we derived for the given problem statement:\
Feed to three units is split into three streams: 𝐹_𝐴, 𝐹_𝐵 and 𝐹_𝐶. <br>
Two products are produced: 𝑃_1 and 𝑃_2, the yield in weight percent by unit and the value of feed and product streams are given below:
| Yield (wt %) | Unit A | Unit B | Unit C |
|--------------|--------|--------|--------|
| P₁           | 40     | 30     | 50     |
| P₂           | 60     | 70     | 50     |


| Stream | Value ($/kg) |
|--------|--------------|
| F      | 0.40         |
| P₁     | 0.60         |
| P₂     | 0.30         |


We have some capacity limitations:\
    1. The plant can handle a maximum feed 𝐹 of 10,000 kg/day\
    2. Each unit (A, B and C) cannot handle more than 5,000 kg/day\
    3. The maximum demand of 𝑃_1 and 𝑃_2 are 4,000 kg/day and 5,000 kg/day, respectively and anything extra that is produced is likely to remain unsold

Determine the optimal values of 𝐹_𝐴, 𝐹_𝐵 and 𝐹_𝐶 that maximize the daily profit of the plant?


We derived the following formulation in the lecture:
\begin{align*}
\max & \ 0.6 P_1 + 0.3 P_2 - 0.4F \\
\text{s.t.} & \ F = F_A + F_B + F_C \\
& \ P_1 = 0.4F_A + 0.3F_B + 0.5F_C \\
& \ P_2 = 0.6F_A + 0.7F_B + 0.5F_C \\
& \ F \leq 10000 \\
& \ F_A \leq 5000 \\
& \ F_B \leq 5000 \\
& \ F_C \leq 5000 \\
& \ P_1 \leq 4000 \\
& \ P_2 \leq 7000 \\
& \ F,F_A,F_B,F_C,P_1,P_2 \geq 0
\end{align*}

Now we will implement this formulation to solve this linear programming problem using SciPy.

Let us import the relevant libraries first.

In [1]:
import numpy as np
from scipy.optimize import linprog

Now we will define the objective function based on the variables in our problem formulation.

In [2]:
# Defining the vector of variables x
# x := [F, F_A, F_B, F_C, P1, P2]
# Coefficients of the objective function (maximize 0.6*P1 + 0.3*P2 - 0.4*F)
# Note: linprog does minimization, so we need to negate the coefficients
c = np.array([0.4, 0, 0, 0, -0.6, -0.3])  # Negated coefficients of F, P1, P2

Now we will define the matrix of coefficients of all the inequality constraints in our formulation

In [4]:
# Coefficients of the inequality constraints (Ax <= b)
A_ub = np.array([
    [1, 0, 0, 0, 0, 0],   # F <= 10000
    [0, 1, 0, 0, 0, 0],   # F_A <= 5000
    [0, 0, 1, 0, 0, 0],   # F_B <= 5000
    [0, 0, 0, 1, 0, 0],   # F_C <= 5000
    [0, 0, 0, 0, 1, 0],   # P1 <= 4000
    [0, 0, 0, 0, 0, 1],   # P2 <= 7000
])

b_ub = np.array([
    10000, # F <= 10000
    5000,  # F_A <= 5000
    5000,  # F_B <= 5000
    5000,  # F_C <= 5000
    5000,  # P1 <= 4000
    7000,  # P2 <= 7000
])

Now we will define the matrix of coefficients of all the equality constraints in our formulation

In [5]:
# Coefficients of the equality constraints (A_eq x = b_eq)
A_eq = np.array([
    [1, -1, -1, -1, 0, 0],  # F = F_A + F_B + F_C
    [0, -0.4, -0.3, -0.5, 1, 0],  # P1 = 0.4*F_A + 0.3*F_B + 0.5*F_C
    [0, -0.6, -0.7, -0.5, 0, 1],  # P2 = 0.6*F_A + 0.7*F_B + 0.5*F_C
])

b_eq = np.array([0, 0, 0])  # Right-hand side of equality constraints

Now we will specify the bounds on alll the variables in the formulation

In [6]:
# Bounds for each variable (F, F_A, F_B, F_C, P1, P2)
x_bounds = (0, None)  # all variables are >= 0
bounds = [x_bounds] * 6

Now we will solve the formulation using the linear programming functionality of SciPy

In [7]:
# Solve the linear program
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

Display the results.

In [8]:
# Display the results
print("Optimal value:", -result.fun)  # negate the result to get the maximized value
print("Optimal solution:")
print("F =", result.x[0])
print("F_A =", result.x[1])
print("F_B =", result.x[2])
print("F_C =", result.x[3])
print("P1 =", result.x[4])
print("P2 =", result.x[5])

Optimal value: 350.0
Optimal solution:
F = 10000.0
F_A = 5000.0
F_B = 0.0
F_C = 5000.0
P1 = 4500.0
P2 = 5500.0


Inspecting the solution in more detail.

In [9]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -350.0
              x: [ 1.000e+04  5.000e+03  0.000e+00  5.000e+03  4.500e+03
                   5.500e+03]
            nit: 0
          lower:  residual: [ 1.000e+04  5.000e+03  0.000e+00  5.000e+03
                              4.500e+03  5.500e+03]
                 marginals: [ 0.000e+00  0.000e+00  1.000e-02  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 4.000e-01 -6.000e-01 -3.000e-01]
        ineqlin:  residual: [ 0.000e+00  0.000e+00  5.000e+03  0.000e+00
                              5.0